In [11]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

### 데이터 로드

In [12]:
ohlcv_df_raw = pd.read_csv('ohlcv_df_raw.csv', dtype = {'ticker':str})
ticker_list = ohlcv_df_raw['ticker'].unique()

In [13]:
ohlcv_df_raw

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market
0,2021-01-04,81000,84400,80200,83000,38655276,2.469136,005930,삼성전자,KOSPI
1,2021-01-05,81600,83900,81600,83900,35335669,1.084337,005930,삼성전자,KOSPI
2,2021-01-06,83300,84500,82100,82200,42089013,-2.026222,005930,삼성전자,KOSPI
3,2021-01-07,82800,84200,82700,82900,32644642,0.851582,005930,삼성전자,KOSPI
4,2021-01-08,83300,90000,83000,88800,59013307,7.117008,005930,삼성전자,KOSPI
...,...,...,...,...,...,...,...,...,...,...
68156,2023-10-24,10170,10300,9870,10280,220969,1.280788,137310,에스디바이오센서,KOSPI
68157,2023-10-25,10270,10420,10100,10310,202518,0.291829,137310,에스디바이오센서,KOSPI
68158,2023-10-26,10100,10130,9800,9820,406661,-4.752667,137310,에스디바이오센서,KOSPI
68159,2023-10-27,9900,9970,9640,9690,283074,-1.323829,137310,에스디바이오센서,KOSPI


In [14]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for ticker_nm in ticker_list:
    df_raw = ohlcv_df_raw[ohlcv_df_raw['ticker'] == ticker_nm].reset_index(drop = True)
    
    ######################################################################
    # 보조지표
    ###################################################################### 

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD 
    macd = MACD(close=df_raw['close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']
    
    
    ######################################################################
    # 보조지표 분석
    ###################################################################### 
    df_raw_anal = df_raw[['date','ticker', 'close']]
    
    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]
    

    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'
    
    
    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0) 

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0) 
    
    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'
    
    
    # 볼린저밴드
    down_reg_sq = df_raw['upper'] - df_raw['close'] 
    top_reg_sq  = df_raw['lower'] - df_raw['close'] 

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]
    
    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[ascending_sq,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[descending_sq,'Bollinger_band'] = '상향회귀(매수)'
    
    
    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'    
    
    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]
    
    
    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])
    
    print(ticker_nm)

/tmp/ipykernel_22019/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_22019/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_22019/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

005930
373220
000660


/tmp/ipykernel_22019/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'RSI'] = '-'
/tmp/ipykernel_22019/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_22019/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

207940
005380


/tmp/ipykernel_22019/984985259.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'Bollinger_band'] = '-'
/tmp/ipykernel_22019/984985259.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'
/tmp/ipykernel_22019/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

...

/tmp/ipykernel_22019/984985259.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'
/tmp/ipykernel_22019/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'RSI'] = '-'
/tmp/ipykernel_22019/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

030000
036460
008930


/tmp/ipykernel_22019/984985259.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'Bollinger_band'] = '-'
/tmp/ipykernel_22019/984985259.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'
/tmp/ipykernel_22019/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

002790
139480
012750
023530
137310


/tmp/ipykernel_22019/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_22019/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_22019/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [15]:
df_raw_total = df_raw_total.reset_index(drop = True)

In [16]:
df_raw_total['MA120']

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
68156    13734.166667
68157    13636.750000
68158    13546.500000
68159    13453.916667
68160    13359.833333
Name: MA120, Length: 68161, dtype: float64

### 등락률 계산

In [17]:
diff_date_list = [30, 90, 180, 240, 365]
now = datetime.now()
set_date = now.strftime('%Y-%m-%d')

ohlcv_df_raw['date'] = pd.to_datetime(ohlcv_df_raw['date'])


df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] == '2023-10-06'].groupby("ticker")['date'].max().reset_index()
df1_set = df1_set[['ticker', 'date']]
df1_set = pd.merge(df1_set, ohlcv_df_raw[['ticker', 'date', 'close', 'corp_name']],
              on = ['ticker', 'date'], 
              how = 'left')

    
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, ohlcv_df_raw[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}']    

In [18]:
df1_set_2 = df1_set[['ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]

In [19]:
df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by =per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per_total = pd.concat([df_per_total, df_per])

In [20]:
df_per_total = df_per_total.drop_duplicates()

In [21]:
df_per_total.sort_values(by = 'per_30', ascending = True)

,ticker,per_30,per_90,per_180,per_240,per_365,corp_name,type,rank
63,051900,-0.289954,-0.307350,-0.470187,-0.536513,-0.390196,LG생활건강,per_30,1
63,051900,-0.289954,-0.307350,-0.470187,-0.536513,-0.390196,LG생활건강,per_180,2
63,051900,-0.289954,-0.307350,-0.470187,-0.536513,-0.390196,LG생활건강,per_240,2
63,051900,-0.289954,-0.307350,-0.470187,-0.536513,-0.390196,LG생활건강,per_365,4
0,000100,-0.277215,-0.215659,-0.023932,0.077358,-0.001818,유한양행,per_30,2
40,018880,-0.246285,-0.155767,-0.239014,-0.255765,-0.089744,한온시스템,per_30,3
9,003670,-0.239583,-0.474280,-0.245199,0.089552,0.280702,포스코퓨처엠,per_30,4
9,003670,-0.239583,-0.474280,-0.245199,0.089552,0.280702,포스코퓨처엠,per_90,1
20,008770,-0.231353,-0.176152,-0.277051,-0.224490,-0.064615,호텔신라,per_30,5
61,047050,-0.207353,-0.365882,0.928444,1.384956,1.610169,포스코인터내셔널,per_365,1
